In [18]:
# =============================================================
# 06_train_image_model.ipynb — Full EfficientNetB3 model
# =============================================================
import os
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


c:\Users\Negar\anaconda3\envs\crlm\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [19]:
# -------------------------------------------------------------
# Environment setup
# -------------------------------------------------------------
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detected: {gpus}")
else:
    print("⚠️ No GPU detected — training will use CPU (much slower).")

# Parameters
IMG_SIZE = (300, 300)        # EfficientNetB3 default input
BATCH_SIZE = 16
EPOCHS = 30
SEED = 42

DATA_DIR = Path("data/processed/images/all_slices")
INDEX = Path("data/processed/images/all_index.csv")
SPLIT_DIR = Path("data/splits")
CLINICAL_PATH = Path("data/processed/clinical/clinical_features_with_id.csv")
SAVE_DIR = Path("results/images")
SAVE_DIR.mkdir(parents=True, exist_ok=True)


TensorFlow version: 2.12.0
⚠️ No GPU detected — training will use CPU (much slower).


In [20]:
# -------------------------------------------------------------
# Load datasets + splits
# -------------------------------------------------------------
df = pd.read_csv(INDEX)
train_ids = pd.read_csv(SPLIT_DIR / "train_series.csv")["series_id"].tolist()
val_ids   = pd.read_csv(SPLIT_DIR / "val_series.csv")["series_id"].tolist()
test_ids  = pd.read_csv(SPLIT_DIR / "test_series.csv")["series_id"].tolist()

def assign_split(sid):
    if sid in train_ids: return "train"
    if sid in val_ids:   return "val"
    if sid in test_ids:  return "test"
    return "ignore"

df["split"] = df["series_id"].map(assign_split)
df = df[df["split"] != "ignore"]

print("🖼 Total image slices:", len(df))
print(df["split"].value_counts())


🖼 Total image slices: 27822
split
train    19603
test      4166
val       4053
Name: count, dtype: int64


In [21]:
# -------------------------------------------------------------
# Merge with clinical metastasis labels
# -------------------------------------------------------------
clinical = pd.read_csv(CLINICAL_PATH)
clinical.columns = clinical.columns.str.strip()
meta_col = next((c for c in clinical.columns if "metastasis" in c.lower()), None)
if meta_col is None:
    raise SystemExit("❌ No metastasis column found in clinical data!")

merged = df.merge(clinical[["patient_id", meta_col]], on="patient_id", how="left")

possible_cols = [c for c in merged.columns if "metastasis" in c.lower()]
meta_col_final = possible_cols[0]
merged = merged.rename(columns={meta_col_final: "metastasis_status"})

merged["label"] = merged["metastasis_status"].replace({
    1: 1, 0: 0,
    "yes": 1, "metastatic": 1, "positive": 1,
    "no": 0, "non-metastatic": 0, "negative": 0
})
merged["label"] = pd.to_numeric(merged["label"], errors="coerce").fillna(0).astype(int)

print(f"✅ Labels assigned: {merged['label'].sum()} metastatic, {len(merged)-merged['label'].sum()} non-metastatic")
print("📊 Label distribution:\n", merged['label'].value_counts())


✅ Labels assigned: 1195 metastatic, 26627 non-metastatic
📊 Label distribution:
 label
0    26627
1     1195
Name: count, dtype: int64


In [22]:
# -------------------------------------------------------------
# TensorFlow dataset builders
# -------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    return tf.cast(img, tf.float32) / 255.0

def make_dataset(subdf, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((subdf["slice_path"].values, subdf["label"].values))
    ds = ds.map(lambda p, y: (decode_img(p), y), num_parallel_calls=AUTOTUNE)
    if augment:
        ds = ds.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
        ds = ds.map(lambda x, y: (tf.image.random_brightness(x, 0.15), y))
    if shuffle:
        ds = ds.shuffle(2048, seed=SEED)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_dataset(merged[merged.split=="train"], augment=True)
val_ds   = make_dataset(merged[merged.split=="val"])
test_ds  = make_dataset(merged[merged.split=="test"], shuffle=False)

print(f"✅ Datasets ready — Train: {len(merged[merged.split=='train'])}, Val: {len(merged[merged.split=='val'])}, Test: {len(merged[merged.split=='test'])}")


✅ Datasets ready — Train: 19603, Val: 4053, Test: 4166


In [ ]:
# -------------------------------------------------------------
# Clear session before building a big model
# -------------------------------------------------------------
tf.keras.backend.clear_session()

In [23]:
# -------------------------------------------------------------
# Model definition — EfficientNetB3
# -------------------------------------------------------------
base = tf.keras.applications.EfficientNetB3(
    include_top=False,
    input_shape=IMG_SIZE + (3,),
    weights="imagenet",
    pooling="avg"
)
base.trainable = True   # fine-tune all layers (can freeze first N later)

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base(inputs, training=True)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

model.summary()


43941136/43941136 [==============================] - 12s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 dropout (Dropout)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 1)                 1537      
                                                                 
Total params: 10,785,072
Trainable params: 10,697,769
Non-trainable params: 87,303
_________________________________________________________________


In [ ]:
# -------------------------------------------------------------
# Callbacks (with JSON-safe and weight-only checkpoints)
# -------------------------------------------------------------
import json
import numpy as np

def safe_float(x):
    """Convert any metric value (tensor/ndarray) to plain float."""
    try:
        if hasattr(x, "numpy"):
            x = x.numpy()
        if isinstance(x, (np.ndarray, list, tuple)):
            return float(np.mean(x))
        return float(x)
    except Exception:
        return 0.0

class SafeJSONCallback(tf.keras.callbacks.Callback):
    """Prevent JSON serialization errors in Keras history."""
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            for k, v in logs.items():
                logs[k] = safe_float(v)

class SafeReduceLROnPlateau(tf.keras.callbacks.ReduceLROnPlateau):
    def on_epoch_end(self, epoch, logs=None):
        logs = {k: safe_float(v) for k, v in (logs or {}).items()}
        super().on_epoch_end(epoch, logs)

# Weight-only checkpoint file (robust across TF versions)
checkpoint_path = SAVE_DIR / "best_image_model.weights.h5"

callbacks = [
    SafeJSONCallback(),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_auc", patience=5, mode="max", restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_path),
        monitor="val_auc", save_best_only=True,
        save_weights_only=True,   # ✅ prevents broken .keras files
        mode="max", verbose=1),
    SafeReduceLROnPlateau(
        monitor="val_auc", factor=0.3, patience=3, verbose=1, mode="max"),
]


In [ ]:
# -------------------------------------------------------------
# Resume from checkpoint if available
# -------------------------------------------------------------
if checkpoint_path.exists():
    print(f"🔁 Found existing checkpoint at {checkpoint_path}, resuming training (loading weights only)...")
    model.load_weights(checkpoint_path)
else:
    print("🚀 Starting new training run from scratch...")

# -------------------------------------------------------------
# Determine last completed epoch
# -------------------------------------------------------------
history_path = SAVE_DIR / "training_history_best.csv"
initial_epoch = 0
if history_path.exists():
    hist = pd.read_csv(history_path)
    initial_epoch = len(hist)
    print(f"⏩ Resuming training from epoch {initial_epoch} of {EPOCHS}")


In [ ]:
# -------------------------------------------------------------
# Train
# -------------------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    verbose=1
)


🚀 Starting full model training...
Epoch 1/30
1017/1226 [=======================>......] - ETA: 48:30 - loss: 0.0181 - accuracy: 0.9950 - auc: 0.9958 - precision: 0.9255 - recall: 0.9305

KeyboardInterrupt: 

In [ ]:
# -------------------------------------------------------------
# Evaluate
# -------------------------------------------------------------
results = model.evaluate(test_ds, return_dict=True)
print("\n✅ Final Test Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

pd.DataFrame(history.history).to_csv(SAVE_DIR / "training_history_best.csv", index=False)


In [ ]:
# -------------------------------------------------------------
# Save model and embeddings
# -------------------------------------------------------------
print("\n💾 Saving model and feature embeddings...")

# Save in pure TensorFlow SavedModel format (robust & safe)
tf.saved_model.save(model, str(SAVE_DIR / "baseline_image_model_full_tf"))
print("✅ Model saved successfully (TensorFlow SavedModel format).")

# Extract feature embeddings
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
all_paths = merged["slice_path"].tolist()
batch_size = 32
embeddings = []

for i in range(0, len(all_paths), batch_size):
    batch_imgs = [decode_img(p) for p in all_paths[i:i+batch_size]]
    batch_tensor = tf.stack(batch_imgs)
    emb = feature_extractor(batch_tensor).numpy()
    embeddings.append(emb)

embeddings = np.vstack(embeddings)
np.save(SAVE_DIR / "image_embeddings_full.npy", embeddings)
pd.DataFrame({
    "slice_path": all_paths,
    "label": merged["label"].tolist()
}).to_csv(SAVE_DIR / "image_embeddings_index_full.csv", index=False)

print(f"✅ Embeddings saved: {embeddings.shape} → {SAVE_DIR/'image_embeddings_full.npy'}")
print("🎯 Training complete.")